![JohnSnowLabs](https://sparknlp.org/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp/blob/master/examples/python/transformers/openvino/HuggingFace_OpenVINO_in_Spark_NLP_XlmRoBertaForZeroShotClassification.ipynb)

# Import OpenVINO XlmRoBertaForZeroShotClassification models from HuggingFace 🤗 into Spark NLP 🚀

This notebook provides a detailed walkthrough on optimizing and exporting XlmRoBertaForZeroShotClassification models from HuggingFace for use in Spark NLP, leveraging the various tools provided in the [Intel OpenVINO toolkit](https://www.intel.com/content/www/us/en/developer/tools/openvino-toolkit/overview.html) ecosystem.

Let's keep in mind a few things before we start 😊

- OpenVINO support was introduced in  `Spark NLP 5.4.0`, enabling high performance inference for models. Please make sure you have upgraded to the latest Spark NLP release.
- You can import models for XlmRoBertaForZeroShotClassification from XlmRoBertaForZeroShotClassification   and they have to be in `Zero-Shot Classification` category.

## 1. Export and Save the HuggingFace model

- Let's install `transformers` and `openvino` packages with other dependencies. You don't need `openvino` to be installed for Spark NLP, however, we need it to load and save models from HuggingFace.
- We lock `transformers` on version `4.41.2`. This doesn't mean it won't work with the future releases, but we wanted you to know which versions have been tested successfully.

In [1]:
!pip install -q --upgrade transformers==4.34.1
!pip install -q --upgrade openvino==2024.3
!pip install -q --upgrade optimum-intel==1.18.3
!pip install -q --upgrade onnx==1.12.0
!pip install --upgrade huggingface-hub


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.5/121.5 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 24.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 20.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 9.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
accelerate 0.34.2 requires huggingface-hub>=0.21.0, but you have huggingface-hub 0.17.3 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 MB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.4/223.4 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.3/474.3 kB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.5/421.5 kB 20.9 MB/s eta 0:00:

[Optimum Intel](https://github.com/huggingface/optimum-intel?tab=readme-ov-file#openvino) is the interface between the Transformers library and the various model optimization and acceleration tools provided by Intel. HuggingFace models loaded with optimum-intel are automatically optimized for OpenVINO, while being compatible with the Transformers API.
- To load a HuggingFace model directly for inference/export, just replace the `AutoModelForXxx` class with the corresponding `OVModelForXxx` class. We can use this to import and export OpenVINO models with `from_pretrained` and `save_pretrained`.
- By setting `export=True`, the source model is converted to OpenVINO IR format on the fly.
- We'll use [symanto/xlm-roberta-base-snli-mnli-anli-xnli](https://huggingface.co/symanto/xlm-roberta-base-snli-mnli-anli-xnli) model from HuggingFace as an example and load it as a `OVModelForSequenceClassification`, representing an OpenVINO model.
- In addition to the OVModelForSequenceClassification model, we also need to save the `AutoTokenizer`. This is the same for every model, these are assets (saved in `/assets`) needed for tokenization inside Spark NLP.

In [2]:
from optimum.intel import OVModelForSequenceClassification
from transformers import AutoTokenizer

MODEL_NAME = "symanto/xlm-roberta-base-snli-mnli-anli-xnli"
EXPORT_PATH = f"ov_models/{MODEL_NAME}"

ov_model = OVModelForSequenceClassification.from_pretrained(MODEL_NAME, export=True)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

# Save the OpenVINO model
ov_model.save_pretrained(EXPORT_PATH)
tokenizer.save_pretrained(EXPORT_PATH)

# Create directory for assets and move the tokenizer files.
# A separate folder is needed for Spark NLP.
!mkdir {EXPORT_PATH}/assets

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:90: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/921 [00:00<?, ?B/s]

Framework not specified. Using pt to export the model.


pytorch_model.bin:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/398 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Using framework PyTorch: 2.4.0+cu121
Overriding 1 configuration item(s)
	- use_cache -> False
Compiling the model to CPU ...


In [3]:
# get label2id dictionary
labels = ov_model.config.id2label
# sort the dictionary based on the id
labels = [value for key,value in sorted(labels.items(), reverse=False)]

with open(EXPORT_PATH + '/assets/labels.txt', 'w') as f:
    f.write('\n'.join(labels))

In [4]:
!mv {EXPORT_PATH}/sentencepiece.bpe.model  {EXPORT_PATH}/assets

## Import and Save XlmRoBertaForZeroShotClassification in Spark NLP


- Let's install and setup Spark NLP in Google Colab
- This part is pretty easy via our simple script

In [ ]:
! wget http://setup.johnsnowlabs.com/colab.sh -O - | bash

--2023-09-29 19:41:03--  http://setup.johnsnowlabs.com/colab.sh
Resolving setup.johnsnowlabs.com (setup.johnsnowlabs.com)... 51.158.130.125
Connecting to setup.johnsnowlabs.com (setup.johnsnowlabs.com)|51.158.130.125|:80... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh [following]
--2023-09-29 19:41:04--  https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp/master/scripts/colab_setup.sh
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.110.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1191 (1.2K) [text/plain]
Saving to: ‘STDOUT’

-                   100%[===================>]   1.16K  --.-KB/s    in 0s      

2023-09-29 19:41:04 (106 MB/s) - written to stdout [1191/1191]

Installing 

Let's start Spark with Spark NLP included via our simple `start()` function

In [ ]:
import sparknlp
# let's start Spark with Spark NLP
spark = sparknlp.start()

print("Apache Spark version: {}".format(spark.version))

Apache Spark version: 3.2.3


- Let's use `loadSavedModel` functon in `XlmRoBertaForZeroShotClassification` which allows us to load TensorFlow model in SavedModel format
- Most params can be set later when you are loading this model in `XlmRoBertaForZeroShotClassification` in runtime like `setMaxSentenceLength`, so don't worry what you are setting them now
- `loadSavedModel` accepts two params, first is the path to the TF SavedModel. The second is the SparkSession that is `spark` variable we previously started via `sparknlp.start()`
- NOTE: `loadSavedModel` accepts local paths in addition to distributed file systems such as `HDFS`, `S3`, `DBFS`, etc. This feature was introduced in Spark NLP 4.2.2 release. Keep in mind the best and recommended way to move/share/reuse Spark NLP models is to use `write.save` so you can use `.load()` from any file systems natively.


In [ ]:
from sparknlp.annotator import *
from sparknlp.base import *

zero_shot_classifier = XlmRoBertaForZeroShotClassification.loadSavedModel(
    EXPORT_PATH,
    spark
    )\
    .setInputCols(["document", "token"]) \
    .setOutputCol("class") \
    .setCandidateLabels(["urgent", "mobile", "travel", "movie", "music", "sport", "weather", "technology"])

- Let's save it on disk so it is easier to be moved around and also be used later via `.load` function

In [ ]:
zero_shot_classifier.write().overwrite().save("./{}_spark_nlp_openvino".format(EXPORT_PATH))

Awesome 😎  !

This is your XlmRoBertaForZeroShotClassification model from HuggingFace 🤗  loaded and saved by Spark NLP 🚀

In [ ]:
! ls -l {MODEL_NAME}_spark_nlp_openvino

total 429464
-rw-r--r-- 1 root root 439759046 Sep 29 19:42 bert_classification_onnx
drwxr-xr-x 4 root root      4096 Sep 29 19:42 fields
drwxr-xr-x 2 root root      4096 Sep 29 19:42 metadata


Now let's see how we can use it on other machines, clusters, or any place you wish to use your new and shiny XlmRoBertaForZeroShotClassification model 😊

In [ ]:
zero_shot_classifier_loaded = XlmRoBertaForZeroShotClassification.load("./{}_spark_nlp_openvino".format(EXPORT_PATH))\
  .setInputCols(["document",'token'])\
  .setOutputCol("class")

You can see what labels were used to train this model via `getClasses` function:

In [ ]:
# .getClasses was introduced in spark-nlp==3.4.0
zero_shot_classifier_loaded.getClasses()

['NEU', 'POS', 'NEG']

This is how you can use your loaded classifier model in Spark NLP 🚀 pipeline:

In [ ]:
from sparknlp.base import *
from sparknlp.annotator import *
from pyspark.ml import Pipeline, PipelineModel

document_assembler = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

tokenizer = Tokenizer().setInputCols("document").setOutputCol("token")

pipeline = Pipeline(stages=[
    document_assembler,
    tokenizer,
    zero_shot_classifier_loaded
])

text = [["I have a problem with my iphone that needs to be resolved asap!!"],
        ["Last week I upgraded my iOS version and ever since then my phone has been overheating whenever I use your app."],
        ["I have a phone and I love it!"],
        ["I really want to visit Germany and I am planning to go there next year."],
        ["Let's watch some movies tonight! I am in the mood for a horror movie."],
        ["Have you watched the match yesterday? It was a great game!"],
        ["We need to harry up and get to the airport. We are going to miss our flight!"]]

# create a DataFrame in PySpark
inputDataset = spark.createDataFrame(text, ["text"])
model = pipeline.fit(inputDataset)
model.transform(inputDataset).select("class.result").show()

+------------------+------+
|              text|result|
+------------------+------+
|Te quiero. Te amo.| [POS]|
+------------------+------+



That's it! You can now go wild and use hundreds of `XlmRoBertaForZeroShotClassification` models from HuggingFace 🤗 in Spark NLP 🚀
